# disney movie data 

# including libraries

In [3]:
from bs4 import BeautifulSoup as bs
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import re
import json


# function defination

In [4]:
def money_conversion(money):        #converting money into float numbers
    reg_i = re.compile(r'\d+\.?\d+') #creating regex object for numerical like 560.009
    reg_mil = re.compile(r'million') #creating regex for eord million
    reg_bil = re.compile(r'billion')
    reg_thou = re.compile(r'thousand')
    a = reg_i.search(money)  # searching money string like budget or box office collection
    b = reg_mil.search(money)
    c = reg_bil.search(money)
    d = reg_thou.search(money)
    if a:  # if there is word million than multiplication into million
        a = reg_i.search(money).group()
        if b:
            a = float(a)*1000000
        elif c:
            a = float(a)*1000000000
        elif d:
            a = float(a)*1000
        else:
            pass
    else:
        return 'no value' # if there is no value for money
    return float(a)
def min_to_int(running_time): #converting minute into ineger
    if running_time == 'N/A': # if no run time do nothing
        return None
    if isinstance(running_time, list): # if run time is list choose char before frst space from frst lielement
        return int(running_time[0].split(' ')[0])
    else: #otherwise use one elements char before frst space
        return int(running_time.split(' ')[0])
    
def get_content_row(row):
    ul_term = row.find('ul') # if there is ul term we need to add all text in value
    if ul_term != None:
        multi = []
        li_term = row.find_all('li')
        for term in li_term:
            result = term.get_text().replace('\xa0', ' ')
            multi.append(result)
            mul = ''
            for s in multi:
                mul += str(s) + ', ' 
        return mul  
    else: # when there is no ul term key will have single value
        return row.find('td').get_text().replace('\xa0', ' ') # removing wierd latters
def clean_tags(soup):# clean up [1] this type of tags from diffrent fields
    for tag in soup.find_all('sup'):
        tag.decompose()
        
def get_info(url):      # getting whole dictionary for a single movie 
    session = requests.Session() # session used for retry 
    retry = Retry(connect=3, backoff_factor=0.5) 
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    r = session.get(url) # get url for a movie
    
    soup = bs(r.content)
    clean_tags(soup)
    info = soup.find(class_='infobox vevent')
    info_tr = info.find_all('tr')
    movie_info = {}
    for index, row in enumerate(info_tr):
        if index == 0:
            movie_info['title']= row.find('th').get_text()
        elif index == 1:
            continue
        else:
            if row.find('b'):
                sub_key = row.find_all('b')
                for sub in sub_key:
                    direc = sub.get_text()
                    sub_value = get_content_row(row)
                    movie_info[direc] = sub_value 
            else:
                content_key = row.find('th').get_text()
                content_value = get_content_row(row)
                movie_info[content_key] = content_value

    return movie_info

def save_data(title, data):
    with open(title, 'w', encoding = 'utf-8') as f:
        json.dump(data, f, ensure_ascii = False, indent = 2)

def load_data(title):
    with open(title, encoding = 'utf-8') as f:
        return json.load(f)

# getting information for all the movies of disney wiki

In [5]:
r_main = requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')
soup = bs(r_main.content)
content = soup.prettify()
movies = soup.select('.wikitable.sortable i a')
link_box = []

for i  in range(len(movies)):
    link = 'https://en.wikipedia.org' + str(movies[i]['href'])
    link_box.append(link)
    i +=1


In [ ]:
movie_info_list = []
for i in range(len(link_box)):
    try:
        info = get_info(link_box[i])
        movie_info_list.append(info)
    except AttributeError:
        print()
print(len(movie_info_list))

In [ ]:
for movie in movie_info_list:
    movie['Running time(int)'] = min_to_int(movie.get('Running time', 'N/A'))
for movie in movie_info_list:
    movie['Budget_in_$'] = money_conversion(movie.get('Budget', 'N/A').replace(',',''))
    movie['Box_office_in _$'] = money_conversion(movie.get('Box office', 'N/A').replace(',',''))
    

In [ ]:
save_data('disney_data2.json', movie_info_list)
movie_info_list = load_data('disney_data2.json')

In [ ]:
import pandas as pd
df = pd.DataFrame(movie_info_list)
df.to_csv('disney_movie_sheet.csv')

In [ ]:
df2 = pd.read_csv('disney_movie_sheet.csv')
df2['Box_office_in _$']

In [ ]:
df2 = pd.read_csv('disney_movie_sheet.csv')
if df2.loc[(df2['Budget_in_$'])  & (df2['Box_office_in _$']) ]:
    df2['Profit_loss'] = df2['Box_office_in _$'].astype(float) - df2['Budget_in_$'].astype(float)
else:
    df2['Profit_loss'] = 'budget or boxoffice data is missing'
df2['Profit_loss']